In [1]:
from pprint import pprint
import re

from datasets import load_dataset, Dataset, DatasetDict
import jsonlines as jsonl
from tqdm.auto import tqdm
import trafilatura as trf
from trafilatura.downloads import DEFAULT_HEADERS

In [2]:
DEFAULT_HEADERS["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.42"
DEFAULT_HEADERS["Cookie"] = 'g_state={"i_p":1696723627075,"i_l":1}; nonce=rCmJ3hfi; uid=48a4d2992c52; sid=1:JaxtYqHLLo/OD6rpFuQmo0ibW32H19+RBAX7miF62pbr+N2aUtyhy4NtG8F4whT0; _ga=GA1.1.224746092.1696716348; __cfruid=5b6450bdd31c9b25efb6e54cc8799ef3ae03d9ed-1697943063; xsrf=8079b55f6fd4; _ga_7JY7T788PK=GS1.1.1698277765.10.1.1698277952.0.0.0'

In [3]:
base = "https://www.janeelliottphd.com/post"
slugs = [
    "ageing-and-windfalls",
    "external-validation-is-a-scam",
    "knees",
    "the-undertow",
    "creativity-getting-stuck-and-the-third-door",
    "there-is-no-magic-key",
    "the-intelligence-of-envy",
    "if-no-one-else-had-it-would-you-still-want-it",
    "an-emotion-is-the-physical-registration-of-a-thought",
    "radically-change-your-perspective-in-five-minutes-per-day",
    "why-mindset-matters-for-internal-resistance",
    "we-re-going-to-need-a-smaller-horse",
    "your-brain-s-secret-mission",
    "one-bullsh-t-saying-to-bin-immediately",
    "7-prompts-to-shift-your-story-right-now",
]

posts = [
    trf.fetch_url(f"{base}/{slug}")
    for slug in slugs
]

contents = [
    trf.extract(post)
    for post in tqdm(posts)
]

  0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
contents = [
    dict(
        title=slug.replace("-", " ").capitalize(),
        text=content.replace("\n", "\n\n"),
    )
    for slug, content in zip(
        slugs,
        contents,
    )
]

In [5]:
dataset = Dataset.from_list(contents)

In [6]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)


In [7]:
def to_chatml(row):
    title = row["title"]
    text = row["text"]
    newline = "\n"
    name = "Maya"
    
    system_message = (
        f"{name} is writing essays about life advice."
        " These are based on her own experiences and"
        " what she has learned from talking to others"
        " about the human psychology and how it works."
    )

    information_message = (
        f"## {name}'s Autobiography"
        "\n\n"
        "...I found a form of coaching designed for smart, skeptical people: logical, rigorous, analytical and mind-blowingly effective. It is 100% woo-free, but its results can look and feel like magic. In half a year, I went from not even knowing why I was so miserable to being the calmest person I know, and happier than I thought possible. I learned skills that keep me clear, focused, in tune with myself, moving toward my goals but possessed of a firm belief in my worth no matter my external achievements."
        "In just six months, I learned something profound and essential about how to live better as a human that changed everything for me. And, to my enduring surprise, I learned it from life coaching."
    )

    markdown = f"# {title}{newline}{newline}{text}"

    # Turn into chatml
    chatml = [
        situation(system_message),
        information(information_message),
        me(markdown, name=name),
    ]

    return dict(chatml=chatml)


In [8]:
dataset = dataset.map(to_chatml).remove_columns(["title", "text"])

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [9]:
dataset.push_to_hub("diwank/samantha_blog-chatml", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]